In [6]:
import torch
import torch.nn as nn
from torchvision import transforms as T
from torchvision import models, transforms
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os.path import join as join_path
import numpy as np
from PIL import Image
from tqdm import tqdm

import math
from torch.utils.data import Dataset, DataLoader
from typing import List

from sklearn.metrics import mean_squared_error
from tqdm import tqdm

import math
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

In [7]:
# !wget http://images.cocodataset.org/zips/val2017.zip
# !unzip val2017.zip > /dev/null

# from google.colab import drive
# drive.mount('/content/drive')

# Data

In [4]:
PATH_TO_FOLDER = '../data/'

In [37]:
train_df = pd.read_csv(join_path(PATH_TO_FOLDER, 'train2017.csv'))
val_df = pd.read_csv(join_path(PATH_TO_FOLDER, 'val2017.csv'))
test_df = pd.read_csv(join_path(PATH_TO_FOLDER, 'test2017.csv'))

train_df['img_dir'] = join_path(PATH_TO_FOLDER, 'train2017')
val_df['img_dir'] = join_path(PATH_TO_FOLDER, 'val2017')
test_df['img_dir'] = join_path(PATH_TO_FOLDER, 'test2017')

In [38]:
%%time

def calc_metrics(df, eps=1e-5):
    metrics = list(df.metric.value_counts().index)
    df['before_score_norm'] = 0
    df['after_score_norm'] = 0
    df['gain_percentage_norm'] = 0
    for metric in metrics:
        _min, _max = min(df[df['metric'] == metric]['before_score'].min(), df[df['metric'] == metric]['after_score'].min()), max(df[df['metric'] == metric]['before_score'].max(), df[df['metric'] == metric]['after_score'].max())
        df.loc[df['metric'] == metric, 'before_score_norm'] = (df.loc[df['metric'] == metric, 'before_score'] - _min) / (_max - _min)
        df.loc[df['metric'] == metric, 'after_score_norm'] = (df.loc[df['metric'] == metric, 'after_score'] - _min) / (_max - _min)
    df['gain_percentage_norm'] = (df['after_score_norm'] - df['before_score_norm']) / (df['before_score_norm'] + eps)
    return df

train_dff = calc_metrics(train_df)
val_df = calc_metrics(val_df)
test_df = calc_metrics(test_df)

CPU times: user 400 ms, sys: 0 ns, total: 400 ms
Wall time: 398 ms


In [39]:
def get_slice_df(df, metric_name=None, attack_type=None):
    if metric_name and attack_type:
        return df[(df['metric'] == metric_name) & (df['attack_type'] == attack_type)]
    elif metric_name:
        return df[(df['metric'] == metric_name)]
    elif attack_type:
        return df[(df['metric'] == metric_name)]
    return

class ImageGainDataset(Dataset):
    def __init__(self, images: list,  target: list, transforms=None):
        self.images = images

        self.transforms = transforms
        self.target = target
        assert len(self.images) == len(self.target), "Wrong len"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        filename = self.images[idx]
        image = Image.open(filename).convert('RGB')
        image_np = np.array(image)
        if len(image_np.shape) == 2:
            image_np = image_np[:, :, None]
            image_np = np.concatenate((image_np, image_np, image_np), axis=2)
            image = Image.fromarray(image_np)
        
        if self.transforms:
            image = self.transforms(image)

        target = self.target[idx]
        return image, target

In [215]:
def create_target(df, metric_name: str, attack_name: str, quantiles: List[float], gain_name: str, qs_calculated=None):
    """Сначала выделяем нулевой класс, затем считаем квантили."""
    target = pd.Series(0, index=sorted(list(set(df['img_dir'] + '/' + df['image_name']))), name='target')
    x = get_slice_df(df, metric_name, attack_name)
    index = x['img_dir'] + '/' + x['image_name']
    data = x[gain_name]
    ind = index[data[data < 0].index]
    target[ind] = 0
    data = data[data > 0]
    qs = []
    for i in range(1, len(quantiles)):
        q_left = quantiles[i-1]
        q_right = quantiles[i]
        if qs_calculated is not None:
            q1, q2 = qs_calculated[i-1][1:]
        else:
            q1 = data.quantile(q_left)
            q2 = data.quantile(q_right)
        
        if i == len(quantiles) - 1:
            ind = index[data[(data >= q1) & (data <= q2)].index]
        else:
            ind = index[data[(data >= q1) & (data < q2)].index]
        qs.append([i, q1, q2])
        target[ind] = i
    return target, qs


def create_mos_target(df, metric_attack_dict, quantiles: List[float], gain_name='gain_percentage_norm', qs_calculated=None):
    targets = pd.DataFrame(index=sorted(list(set(df['img_dir'] + '/' + df['image_name']))))
    qs = {}
    for metric_name in metric_attack_dict.keys():
        for attack_name in metric_attack_dict[metric_name]:
            if qs_calculated is not None:
                x = qs_calculated[f'{metric_name}_{attack_name}']
            else:
                x = None
            targets[f'{metric_name}_{attack_name}'], xx = create_target(df, metric_name, attack_name, quantiles, gain_name, x)
            qs[f'{metric_name}_{attack_name}'] = xx
    return targets, qs

In [188]:
metric_attack_dict = {
    'koncept': ['cnn_24_255', 'fgsm_2_255', 'ifgsm_alpha3_eps2_i3', 'uap_26_255'],
    'lin': ['cnn_22_255', 'fgsm_3_255', 'ifgsm_alpha1_eps3_i2', 'uap_20_255'],
    'p2p': ['cnn_8_255', 'fgsm_5_255', 'ifgsm_alpha2_eps5_i3', 'uap_10_255'],
    'spaq': ['cnn_40_255', 'fgsm_5_255', 'ifgsm_alpha1_eps3_i2', 'uap_20_255'],
    'mdtvsfa': ['fgsm_2_255', 'ifgsm_alpha2_eps2_i3', 'uap_9_255'],
}

In [42]:
QUANTILES = [0, .25, .75, 1]

In [ ]:
target_binned_train, _ = create_mos_target(train_df, metric_attack_dict, QUANTILES)
target_train = target_binned_train.mean(1)
target_binned_val, _ = create_mos_target(val_df, metric_attack_dict, QUANTILES)
target_val = target_binned_val.mean(1)
target_binned_test, _ = create_mos_target(test_df, metric_attack_dict, QUANTILES)
target_test = target_binned_test.mean(1)

In [69]:
# plt.figure(figsize=(26, 4))
# plt.subplot(1, 3, 1)
# plt.hist(target_train, bins=100, label="train");
# plt.legend();
# plt.subplot(1, 3, 2)
# plt.hist(target_val, bins=100, label="val");
# plt.legend();
# plt.subplot(1, 3, 3)
# plt.hist(target_test, bins=100, label="test");
# plt.legend();

In [ ]:
_mean = target_train.mean()
_std = target_train.std()
print(_mean, _std)

In [44]:
X_train = (sorted(list(set(train_df['img_dir'] + '/' + train_df['image_name'])))) # sorted(list(set(train_df['image_name'])))
y_train = (target_train[X_train].values - _mean) / _std

X_val = (sorted(list(set(val_df['img_dir'] + '/' + val_df['image_name'])))) # sorted(list(set(val_df['image_name'])))
y_val = (target_val[X_val].values - _mean) / _std

X_test = (sorted(list(set(test_df['img_dir'] + '/' + test_df['image_name'])))) # sorted(list(set(test_df['image_name'])))
y_test = target_test[X_test].values

# Models

In [25]:
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
       values taken from a normal distribution.'''

    classname = m.__class__.__name__
    if classname.find('Linear') != -1: # for every Linear layer in a model
        y = m.in_features
        m.weight.data.normal_(0.0,1/np.sqrt(y)) # m.weight.data shoud be taken from a normal distribution
        m.bias.data.fill_(0) # m.bias.data should be 0
        
def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp

In [26]:
from torch.nn.functional import pad


def collate_padding(batch):
    h = max([b[0].shape[1] for b in batch])
    w = max([b[0].shape[2] for b in batch])
    imgs = torch.zeros(len(batch), 3, h, w)
    targets = torch.zeros(len(batch))
    for i, (im, target) in enumerate(batch):
        new_w = int((w - im.shape[2]) / 2)
        new_h = int((h - im.shape[1]) / 2)
        imgs[i] = pad(im, pad=(new_w, w - new_w - im.shape[2], new_h, h - new_h - im.shape[1]), value=0)
        targets[i] = target
    return imgs, targets

transforms_super_simple = T.Compose([
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])
])

## IRAA

In [28]:
class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, bias=False) # verify bias false
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        # x = self.bn(x)
        x = self.relu(x)
        return x

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pool_kernel_size=None):
        super(CNNBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), padding=(1, 1), bias=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), padding=(1, 1), bias=True)
        
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((pool_kernel_size, pool_kernel_size)) if pool_kernel_size is not None else nn.Identity()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x


class IRAA(nn.Module):
    def __init__(self, n_classes=1, dropout=0.05, pool_kernel_size=None, average_pool_size=1):
        super().__init__()
        self.encoder = nn.Sequential(
            CNNBlock(3, 64, pool_kernel_size=pool_kernel_size),
            nn.Dropout2d(dropout),
            CNNBlock(64, 32, pool_kernel_size=pool_kernel_size),
            nn.Dropout2d(dropout),
            CNNBlock(32, 16, pool_kernel_size=pool_kernel_size),
            nn.AdaptiveAvgPool2d((average_pool_size, average_pool_size)),
        )

        self.flatten = nn.Flatten()
        self.decoder = nn.Sequential(
            nn.Linear(16 * average_pool_size * average_pool_size, 32),
            nn.ReLU(),
            nn.Linear(32, n_classes),
            nn.Sigmoid()       
        )
        self.encoder.apply(weights_init_normal)
        self.decoder.apply(weights_init_normal)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.flatten(x)
        x = self.decoder(x)
        return x * 3


## ResNet18, 50

In [29]:
class ResNet18Model(nn.Module):
    def __init__(self, n_classes=1, pretrained=False):
        super().__init__()
        if pretrained:
            self.encoder = models.resnet18(weights='DEFAULT')
        else:
            self.encoder = models.resnet18(weights=None)
        
        self.decoder = nn.Sequential(
            nn.Linear(1000, n_classes),
            nn.Sigmoid()   
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x * 3
    
class ResNet50Model(nn.Module):
    def __init__(self, n_classes=1, pretrained=False):
        super().__init__()
        if pretrained:
            self.encoder = models.resnet50(weights='DEFAULT')
        else:
            self.encoder = models.resnet50(weights=None)
        
        self.decoder = nn.Sequential(
            nn.Linear(1000, n_classes),
            nn.Sigmoid()   
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x * 3
    
        
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## VGG11

In [30]:
class VGG11(nn.Module):
    def __init__(self, n_classes=1, pretrained=False):
        super().__init__()
        if pretrained:
            self.encoder = models.vgg11(weights='DEFAULT')
        else:
            self.encoder = models.vgg11(weights=None)
        
        self.decoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1000, n_classes),
            nn.Sigmoid()   
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x * 3

## Inception-V3

In [31]:
class Inception3Model(nn.Module):
    def __init__(self, n_classes=1, pretrained=False):
        super().__init__()
        if pretrained:
            self.encoder = models.inception_v3(weights='DEFAULT')
        else:
            self.encoder = models.inception_v3(weights=None)

        self.decoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1000, n_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        if self.training:
            x, _ = self.encoder(x)
        else:
            x = self.encoder(x)
        x = self.decoder(x)
        return x * 3

transform_inception = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## EfficientNet-B2

In [32]:
class EfficientNetB2(nn.Module):
    def __init__(self, n_classes=1, pretrained=False):
        super().__init__()
        if pretrained:
            self.encoder = models.efficientnet_b2(weights='DEFAULT')
        else:
            self.encoder = models.efficientnet_b2(weights=None)
        
        self.decoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(1000, n_classes),
            nn.Sigmoid()   
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x * 3

# Training

In [36]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [95]:
def evaluate(model, dataloader, device):
    model.to(device)
    model.eval()
    all_pred_scores = []
    all_true_scores = []
    with torch.no_grad():
        for batch in tqdm(dataloader, total=len(dataloader), desc="eval"):
            batch_images, batch_labels = batch
            pred_scores = model(batch_images.to(device)).detach().cpu()
            all_pred_scores.extend(pred_scores.numpy())
            all_true_scores.extend(batch_labels.numpy())
            
            torch.cuda.empty_cache()
    
    all_pred_scores = np.array(all_pred_scores).flatten()
    all_true_scores = np.array(all_true_scores).flatten()  
    return all_true_scores, all_pred_scores

## Init model

In [132]:
model = IRAA(dropout=0.1, pool_kernel_size=3, average_pool_size=3)
model = model.to(device)

In [ ]:
# save_model_path = 'iraa.pt'
# model.load_state_dict(torch.load(save_model_path))

In [46]:
collate_fn = collate_padding
# collate_fn = None
current_transforms = transforms_super_simple


In [48]:
batch_size = 16

train_dataset = ImageGainDataset(X_train, y_train, transforms=current_transforms) # transform_resnet, transforms_simple
val_dataset = ImageGainDataset(X_val, y_val, transforms=current_transforms)
test_dataset = ImageGainDataset(X_test, y_test, transforms=current_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=1, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, shuffle=False, collate_fn=collate_fn)

In [49]:
test_dataset = ImageGainDataset(X_test, y_test, transforms=current_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, shuffle=False, collate_fn=collate_fn)

In [92]:
num_epochs = 50
lr = 3e-4
log_name = 'iraa'
loss_fn = nn.MSELoss()
print(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
get_n_params(model)

In [ ]:
best_val_corr = -1000
for epoch in range(num_epochs):
    model = model.to(device)
    ############### TRAIN #####################
    model.train()

    losses = []
    all_pred_scores = []
    all_true_scores = []
    for batch in tqdm(train_dataloader, total=len(train_dataloader), desc=f"epoch: {str(epoch).zfill(3)} | train"):
        optimizer.zero_grad()
        batch_images, batch_labels = batch 
        pred_scores = model(batch_images.to(device))
        pred_scores = (pred_scores - _mean) / _std
        loss = loss_fn(pred_scores, batch_labels.unsqueeze(1).float().to(device))
        loss.backward()
        optimizer.step()
        
        pred_scores = pred_scores.detach().cpu().flatten()

        all_pred_scores.extend(pred_scores.numpy())
        all_true_scores.extend(batch_labels.numpy())

        losses.append(loss.detach().cpu().item())
        
        torch.cuda.empty_cache()
        

    all_pred_scores = np.array(all_pred_scores).flatten()
    all_true_scores = np.array(all_true_scores).flatten()
    train_loss = np.mean(losses)
    train_corr = np.corrcoef(all_pred_scores, all_true_scores)[0, 1]
    
    ############### EVAL ######################
    model.eval()
    
    losses = []
    all_pred_scores = []
    all_true_scores = []
    for batch in tqdm(val_dataloader, total=len(val_dataloader), desc=f"epoch: {str(epoch).zfill(3)} | eval"):
        batch_images, batch_labels = batch
        pred_scores = model(batch_images.to(device))
        pred_scores = (pred_scores - _mean) / _std
        pred_scores = pred_scores.detach().cpu()

        loss = loss_fn(pred_scores, batch_labels.unsqueeze(1).float())
        losses.append(loss.detach().cpu().item())

        all_pred_scores.extend(pred_scores.numpy())
        all_true_scores.extend(batch_labels.numpy())
        
    all_pred_scores = np.array(all_pred_scores).flatten()
    all_true_scores = np.array(all_true_scores).flatten()
    val_loss = np.mean(losses)
    val_corr = np.corrcoef(all_pred_scores, all_true_scores)[0, 1]
    
    with open(join_path('/home/jovyan/work/experiments/logs', f'{log_name}.log'), 'a') as f:
        f.write(f"{epoch},{train_loss},{train_corr},{val_loss},{val_corr}\n")
    
    if val_corr > best_val_corr:
        best_val_corr = val_corr
        torch.save(model.state_dict(), join_path('/home/jovyan/work/experiments/logs', f'{log_name}_best.pt'))
        
    torch.cuda.empty_cache()
        
    print(f"epoch: {str(epoch).zfill(3)} | train_loss: {train_loss:5.3f} | train_corr: {train_corr:5.3f} | val_loss: {val_loss:5.3f} | val_corr: {val_corr:5.3f}")

# Evaluate

In [ ]:
model = IRAA(dropout=0.1, pool_kernel_size=3, average_pool_size=3)
save_model_path = '/home/jovyan/work/experiments/models/IRAA/model_padding.pt'
model.load_state_dict(torch.load(save_model_path))
all_true_scores, all_pred_scores = evaluate(model, test_dataloader, device)

In [119]:
all_true_scores, all_pred_scores = evaluate(model, test_dataloader, device)

eval: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 188/188 [10:16<00:00,  3.28s/it]


In [ ]:
print(np.corrcoef(all_pred_scores, all_true_scores)[0, 1])
print(mean_squared_error(all_pred_scores, all_true_scores))

# Eval Datasets

In [5]:
def collate_padding_no_target(batch):
    max_h = max([b.shape[1] for b in batch])
    max_w = max([b.shape[2] for b in batch])
    h = 2 ** math.ceil(np.log2(max_h))
    w = 2 ** math.ceil(np.log2(max_w))
    imgs = torch.zeros(len(batch), 3, h, w)
    for i, im in enumerate(batch):
        new_w = int((w - im.shape[2]) / 2)
        new_h = int((h - im.shape[1]) / 2)
        imgs[i] = pad(im, pad=(new_w, w - new_w - im.shape[2], new_h, h - new_h - im.shape[1]), value=0)
    return imgs

def collate_padding_new_no_target(batch):
    max_h = max([b.shape[1] for b in batch])
    max_w = max([b.shape[2] for b in batch])
    h = max_h
    w = max_w
    imgs = torch.zeros(len(batch), 3, h, w)

    for i, (im) in enumerate(batch):
        new_w = int((w - im.shape[2]) / 2)
        new_h = int((h - im.shape[1]) / 2)
        imgs[i] = pad(im, pad=(new_w, w - new_w - im.shape[2], new_h, h - new_h - im.shape[1]), value=0)
    return imgs

def predict_only(model, dataloader, device):
    model.to(device)
    model.eval()
    all_pred_scores = []
    with torch.no_grad():
        for batch_images in tqdm(dataloader, total=len(dataloader), desc="eval"):
            pred_scores = model(batch_images.to(device)).detach().cpu()
            all_pred_scores.extend(pred_scores.numpy())
    all_pred_scores = np.array(all_pred_scores).flatten()
    return all_pred_scores

class ImageGainDataset(Dataset):
    def __init__(self, images: list,  transforms=None):
        self.images = images
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        filename = self.images[idx]
        image = Image.open(filename).convert('RGB')
        image_np = np.array(image)
        if len(image_np.shape) == 2:
            image_np = image_np[:, :, None]
            image_np = np.concatenate((image_np, image_np, image_np), axis=2)
            image = Image.fromarray(image_np)            
        
        if self.transforms:
            image = self.transforms(image)
        return image

In [17]:
PATH_TO_FOLDER = '/home/jovyan/work/data'
batch_size = 8

all_names = {
    'nips2017': join_path(PATH_TO_FOLDER, 'NIPS2017/images', '*'),
    'kadid': join_path(PATH_TO_FOLDER, 'KADID10K/kadid10k/images', '*'),
    'liv_in_wild': '/home/jovyan/storage/datasets/subjective/image/LIVE_itW/Images/*.*',
    'tid2013': join_path(PATH_TO_FOLDER, 'TID2013/distorted_images', '*'), 
    'cid2013': join_path(PATH_TO_FOLDER, 'CID2013', "*", "*", "*.jpg"),
    'pipal': join_path(PATH_TO_FOLDER, 'PIPAL_train', "*", "*.bmp"),
    'test2017': join_path(PATH_TO_FOLDER, 'test2017', '*'),
    'div2k': join_path(PATH_TO_FOLDER, 'DIV2K_train_HR', '*.png'), 
}

In [ ]:
model = IRAA(dropout=0.1, pool_kernel_size=3, average_pool_size=3)
save_model_path = '/home/jovyan/work/experiments/models/IRAA/model_padding.pt'
model.load_state_dict(torch.load(save_model_path))
model_name = 'iraa'

for name, im_dir in all_names.items():
    batch_size = 8
    if name in ['nips2017', 'kadid']:
        continue
    if name == 'div2k':
        batch_size = 4
    print(name)
    X = sorted(glob(im_dir))
    dataset = ImageGainDataset(X, transforms=transforms_super_simple)
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=1, shuffle=False, collate_fn=collate_padding_new_no_target)
    pred_scores = predict_only(model, dataloader, device)
    
    save_path = f'/home/jovyan/work/experiments/logs/{model_name}_dataset_hack_{name}.npy'
    with open(save_path, 'wb') as f:
        np.save(f, pred_scores)